 ## Setup library
 tambah library torchsharp dan tambah beberapa direktif untuk mengakses beberapa class dan fungsi yang dibutuhkan

In [24]:
#r "nuget: TorchSharp-cpu"
using TorchSharp;
using static TorchSharp.TensorExtensionMethods;
using static TorchSharp.torch;
using static TorchSharp.torch.nn;
using static TorchSharp.torch.nn.functional;
using Microsoft.DotNet.Interactive.Formatting;
using static TorchSharp.torch.distributions;


var style = TensorStringStyle.Julia;

//untuk formatting saat show isi variabel tensor
Formatter.SetPreferredMimeTypesFor(typeof(torch.Tensor), "text/plain");
Formatter.Register<torch.Tensor>((torch.Tensor x) => x.ToString(style, newLine: "\n"));

Installed Packages TorchSharp-cpu, 0.96.6

### tensors 

struktur data dasar saat menggunakan torchsharp itu adalah tensor, bisa dibayangkan kalau 0D array itu Scalar, 1D array itu vektor, 2D array itu matrix, dan N-dimension array adalah tensor.
berikut adalah cara penggunaan tensor dalam torchsharp:

In [16]:
//inisiasi tensor dengan dimensi 2x3 atau bayangkan seperti tabel dengan 2 baris dan 3 kolom, di isi dengan angka 1
Console.WriteLine("tensor 2x3");
var t = torch.ones(2,3);
t.print();

//tensor dengan dimensi 2 x 3 x 4, di isi dengan angka 0
Console.WriteLine("tensor 2x3x4");
var x = torch.zeros(2,3,4);
x.print();

//tensor 3x3 di isi dengan angka 4
Console.WriteLine("tensor 3x3");
var y = torch.full(3,3,4);
y.print();

//inisiasi tensor dengan dimensi 1x3 
Console.WriteLine("tensor 1x3");
t = torch.empty(1,3);
t.print();


tensor 2x3
[2x3], type = Float32, device = cpu
 1 1 1
 1 1 1

tensor 2x3x4
[2x3x4], type = Float32, device = cpu
[0,..,..] =
 0 0 0 0
 0 0 0 0
 0 0 0 0

[1,..,..] =
 0 0 0 0
 0 0 0 0
 0 0 0 0

tensor 3x3
[3x3], type = Int64, device = cpu
 4 4 4
 4 4 4
 4 4 4

tensor 1x3
[1x3], type = Float32, device = cpu
 6.7041e-39 1.0653e-38 1.0469e-38



In [17]:

//inisiasi tensor dengan tipe data int32
x = torch.zeros(2,6, dtype: torch.int32);
//ubah baris pertama jadi angka 5
for(int i=0;i<6;i++)
    x[0,i] = 5;
x.print();

//ambil baris kedua, masukan ke list
var take = new List<int>();
for(int i=0;i<6;i++)
    take.Add(x[1,i].item<int>());
Console.WriteLine($"Isi list:{take.Count} item");

//inisiasi tensor dengan tipe data kompleks
y = torch.ones(2,6, dtype: torch.complex64);
y.real.print();
y.imag.print();

//isi tensor dengan nilai random (0-1) dengan distribusi normal
t = torch.randn(3,4);
t.print();

//isi tensor dengan nilai random (0-1) dengan distribusi uniform
t = torch.rand(3,4);
t.print();

//isi tensor dengan nilai random dengan tipe integer dan nilai max 100
t = torch.randint(100, (3,4));
t.print();

//isi tensor dengan nilai random dari angka 1-10
t = torch.randperm(11);
t.print();



[2x6], type = Int32, device = cpu
 5 5 5 5 5 5
 0 0 0 0 0 0

Isi list:6 item
[2x6], type = Float32, device = cpu
 1 1 1 1 1 1
 1 1 1 1 1 1

[2x6], type = Float32, device = cpu
 0 0 0 0 0 0
 0 0 0 0 0 0

[3x4], type = Float32, device = cpu
 0.22065 0.63409    0.9958 0.43995
 0.89721 0.20242  0.088088  0.3583
 0.20541 0.42738 -0.059234 -1.0646

[3x4], type = Float32, device = cpu
 0.27736 0.35925 0.65187 0.096936
  0.9982 0.31815 0.10414  0.16131
 0.37161 0.22189 0.86772   0.9104

[3x4], type = Int64, device = cpu
 62 88 14 37
 63 47 51 88
 24 80 43 64

[11], type = Int64, device = cpu
 0 3 5 6 8 2 1 9 7 10 4



In [21]:
//dengan seed yang sama akan menghasilkan angka acak yang sama
var generator1 = new torch.Generator(123);
var generator2 = new torch.Generator(123);

var rnd11 = torch.rand(2,3, generator: generator1);
rnd11.print();
var rnd12 = torch.rand(2,3, generator: generator2);
rnd12.print();



//arange, bikin tensor dengan nilai urut dari 1 - 10
t = torch.arange(1,11);
t.print();
//reshape, ubah dimensi dari 1d menjadi 2x5
t.reshape(2,5).print();
//reshape dengan -1, otomatis terisi dengan sisanya
t.reshape(-1,5).print();
//arange, bikin tensor dengan nilai 2 sampe 5 dengan step 0.2
t = torch.arange(2.0f, 5.0f, step: 0.2f).print();

[2x3], type = Float32, device = cpu
 0.29611  0.51656 0.25167
 0.68856 0.073972 0.86652

[2x3], type = Float32, device = cpu
 0.29611  0.51656 0.25167
 0.68856 0.073972 0.86652

[10], type = Int64, device = cpu
 1 2 3 4 5 6 7 8 9 10

[2x5], type = Int64, device = cpu
 1 2 3 4  5
 6 7 8 9 10

[2x5], type = Int64, device = cpu
 1 2 3 4  5
 6 7 8 9 10

[15], type = Float32, device = cpu
 2 2.2 2.4 2.6 2.8 3 3.2 3.4 3.6 3.8 4 4.2 4.4 4.6 4.8



In [68]:
t = torch.arange(1,11).reshape(2,5);
t.print();
//lihat dimensi tensor
Console.WriteLine("dimensi: " + t.ndim);
Console.WriteLine("jumlah element: " + t.numel());
Console.WriteLine("shape: " + String.Join(',',t.shape));

//swith dari CPU ke CUDA (GPU)
//var x = t.cuda()

//swith dari CUDA (GPU) ke CPU
//t = x.cpu()

//transpose 
t = t.T;
t.print();

//clone
var z = t.clone();
z.print();

//jumlah tensor yang aktif di memory
Console.WriteLine($"total active tensors: {torch.Tensor.TotalCount}");

Console.WriteLine($"Peak Count: {torch.Tensor.PeakCount}");

var b = z.alias(); //reference ke z
b[0,0] = torch.tensor(100);
z.print();

[2x5], type = Int64, device = cpu
 1 2 3 4  5
 6 7 8 9 10

dimensi: 2
jumlah element: 10
shape: 2,5
[5x2], type = Int64, device = cpu
 1  6
 2  7
 3  8
 4  9
 5 10

[5x2], type = Int64, device = cpu
 1  6
 2  7
 3  8
 4  9
 5 10

total active tensors: 2247
Peak Count: 2247
[5x2], type = Int64, device = cpu
 100  6
   2  7
   3  8
   4  9
   5 10




warning CS1701: Assuming assembly reference 'System.Runtime.Numerics, Version=4.1.2.0, Culture=neutral, PublicKeyToken=b03f5f7f11d50a3a' used by 'TorchSharp' matches identity 'System.Runtime.Numerics, Version=6.0.0.0, Culture=neutral, PublicKeyToken=b03f5f7f11d50a3a' of 'System.Runtime.Numerics', you may need to supply runtime policy



In [44]:
//bernouli distribution
var bern = Bernoulli(torch.tensor(0.25f));
var acak1 = bern.sample().item<float>();
var acak2 = bern.sample(2,3);
acak2.print();

//binomial distribution 1/0
var bin = Binomial(torch.tensor(100), torch.tensor(0.5f));
acak1 = bin.sample().item<float>();
acak2 = bin.sample(3,3);
acak2.print();

//multinonmial 1 to N
var mult = Multinomial(100, new float[]{0.1f, 0.5f, 0.1f, 0.9f});
mult.sample().print();
mult.sample(3).print();

//fungsi menghasilkan tensor 2x2 dengan fungsi distribusi yang dinamis
torch.Tensor GenerateNumber(Distribution dist) { return dist.sample(2,2);}

var norm1 = Normal(torch.tensor(0.5f), torch.tensor(0.125f));
var uni = Uniform(torch.tensor(9.0f), torch.tensor(21.0f));
GenerateNumber(uni).print();
GenerateNumber(norm1).print();
GenerateNumber(mult).print();

//category, seperti bernouli tapi bisa support 1 ke N category
var cat = Categorical(torch.tensor(new float[]{0.1f, 0.2f, 0.3f}));
cat.sample(10).print();


[2x3], type = Float32, device = cpu
 0 1 1
 0 0 1

[3x3], type = Float32, device = cpu
 57 51 45
 54 48 59
 48 49 49

[4], type = Float32, device = cpu
 8 32 3 57

[3x4], type = Float32, device = cpu
 12 22 7 59
  6 31 8 55
  8 31 6 55

[2x2], type = Float32, device = cpu
 19.408 16.811
 19.837 16.066

[2x2], type = Float32, device = cpu
 0.67062 0.57671
 0.52696 0.52677

[2x2x4], type = Float32, device = cpu
[0,..,..] =
 8 26 11 55
 3 32  8 57

[1,..,..] =
 9 31 3 57
 6 33 4 57

[10], type = Int64, device = cpu
 1 2 2 0 2 2 0 1 1 1




warning CS1701: Assuming assembly reference 'System.Runtime.Numerics, Version=4.1.2.0, Culture=neutral, PublicKeyToken=b03f5f7f11d50a3a' used by 'TorchSharp' matches identity 'System.Runtime.Numerics, Version=6.0.0.0, Culture=neutral, PublicKeyToken=b03f5f7f11d50a3a' of 'System.Runtime.Numerics', you may need to supply runtime policy

warning CS1701: Assuming assembly reference 'System.Runtime.Numerics, Version=4.1.2.0, Culture=neutral, PublicKeyToken=b03f5f7f11d50a3a' used by 'TorchSharp' matches identity 'System.Runtime.Numerics, Version=6.0.0.0, Culture=neutral, PublicKeyToken=b03f5f7f11d50a3a' of 'System.Runtime.Numerics', you may need to supply runtime policy

warning CS1701: Assuming assembly reference 'System.Runtime.Numerics, Version=4.1.2.0, Culture=neutral, PublicKeyToken=b03f5f7f11d50a3a' used by 'TorchSharp' matches identity 'System.Runtime.Numerics, Version=6.0.0.0, Culture=neutral, PublicKeyToken=b03f5f7f11d50a3a' of 'System.Runtime.Numerics', you may need to supply runt

### Operasi Numerik
Tensor dapat di operasikan secara matematika dengan operator dan fungsi-fungsi built-in torchsharp, berikut adalah contoh-contohnya:

In [76]:
var a = torch.arange(1,11).reshape(2,5);
var b = torch.full(2,5,2);
var c = torch.tensor(3);
//operator, perkalian disini bukan perkalian matrix
var d = a * b + c;
a.print();
b.print();
c.print();
d.print();



[2x5], type = Int64, device = cpu
 1 2 3 4  5
 6 7 8 9 10

[2x5], type = Int64, device = cpu
 2 2 2 2 2
 2 2 2 2 2

[], type = Int32, device = cpu, value = 3
[2x5], type = Int64, device = cpu
  5  7  9 11 13
 15 17 19 21 23

[2x5], type = Int64, device = cpu
  2  4  6  8 10
 12 14 16 18 20




warning CS1701: Assuming assembly reference 'System.Runtime.Numerics, Version=4.1.2.0, Culture=neutral, PublicKeyToken=b03f5f7f11d50a3a' used by 'TorchSharp' matches identity 'System.Runtime.Numerics, Version=6.0.0.0, Culture=neutral, PublicKeyToken=b03f5f7f11d50a3a' of 'System.Runtime.Numerics', you may need to supply runtime policy



In [90]:

//operator matrix
a = torch.arange(1,5).reshape(2,2);
b = torch.full(2,2,1);
a.print();
b.print();
d = a.mm(b);
d.print();

//menggunakan in-place operator, bisa di operasikan secara berantai
var c = torch.arange(2,6).reshape(2,2);
c.print();
// a=a*b+c
a.mul_(b).add_(c);
a.print();

[2x2], type = Int64, device = cpu
 1 2
 3 4

[2x2], type = Int64, device = cpu
 1 1
 1 1

[2x2], type = Int64, device = cpu
 3 3
 7 7

[2x2], type = Int64, device = cpu
 2 3
 4 5

[2x2], type = Int64, device = cpu
 3 5
 7 9

